In [1]:
import pycisTopic
import glob
import os
import pybiomart as pbm
import pandas as pd
import pickle
from pycisTopic.qc import *
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import multiprocess as mp  # for kde multithreading calculation
from multiprocess import Pool

%matplotlib inline
%load_ext lab_black

# Download annotation

In [2]:
!pwd

/lustre1/project/stg_00090/scatac_benchmark/fixedcells_2_cistopic


In [4]:
wdir = "/lustre1/project/stg_00090/scatac_benchmark/fixedcells_2_cistopic"
os.chdir(wdir)

In [5]:
genome = "hg38"

pbm_genome_name_dict = {
    "hg38": "hsapiens_gene_ensembl",
    "hg37": "hsapiens_gene_ensembl",
    "mm10": "mmusculus_gene_ensembl",
    "dm6": "dmelanogaster_gene_ensembl",
}

pbm_host_dict = {
    "hg38": "http://www.ensembl.org",
    "hg37": "http://grch37.ensembl.org/",
    "mm10": "http://nov2020.archive.ensembl.org/",
    "dm6": "http://www.ensembl.org",
}

if os.path.exists(f"annotation.tsv"):
    print(f"Loading cached genome annotation...")
    annotation = pd.read_csv("annotation.tsv", sep="\t", header=0, index_col=0)
else:
    dataset = pbm.Dataset(name=pbm_genome_name_dict[genome], host=pbm_host_dict[genome])

    annot = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
    filter = annot["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
    annot = annot[~filter]
    annot["Chromosome/scaffold name"] = annot["Chromosome/scaffold name"].str.replace(
        r"(\b\S)", r"chr\1"
    )
    annot.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
    annot = annot[annot.Transcript_type == "protein_coding"]
    annot.to_csv("annotation.tsv", sep="\t")

Loading cached genome annotation...


In [6]:
annotation

,Chromosome,Start,Strand,Gene,Transcript_type
57,chrKI270711chr.chr1,24650,-1,NaN,protein_coding
58,chrKI270711chr.chr1,29626,-1,NaN,protein_coding
628,chrKI270734chr.chr1,161852,-1,NaN,protein_coding
629,chrKI270734chr.chr1,161750,-1,NaN,protein_coding
713,chrKI270734chr.chr1,131494,1,NaN,protein_coding
...,...,...,...,...,...
242597,chr1,154257071,1,UBAP2L,protein_coding
242604,chr1,154272589,1,HAX1,protein_coding
242605,chr1,154272609,1,HAX1,protein_coding
242607,chr1,154272629,1,HAX1,protein_coding


In [11]:
fragments_list = sorted(glob.glob("../1_data_repository/fixedcells_fragments/*.tsv.gz"))
fragments_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".fragments.tsv.gz")[0]
    fragments_dict[sample] = fragments_file

In [12]:
fragments_dict

{'BIO_ddseq_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_1.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_2.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_3.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_3.FIXEDCELLS.fragments.tsv.gz',
 'BIO_ddseq_4.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BIO_ddseq_4.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_1.FIXEDCELLS.fragments.tsv.gz',
 'BRO_mtscatac_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/BRO_mtscatac_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_1.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xmultiome_2.FIXEDCELLS': '../1_data_repository/fixedcells_fragments/CNA_10xmultiome_2.FIXEDCELLS.fragments.tsv.gz',
 'CNA_10xv11_1.FIXEDCELLS': '../1_data_repository/fixedcells_fragmen

The regions we will count in. Here, I count everything in the human ENCODE SCREEN regions. You can also add a sample-specific region set, e.g. sample-specific peaks called on the sample's bams.

In [10]:
region = "../0_resources/regions/V2.hg38-rDHS-Unfiltered.blacklisted.bed"

Now, make a sub dictionary of all samples within the fragments dict that have not been run yet (good for resuming a stopped cistopic run):

In [9]:
cistopic_qc_out = os.path.join(wdir, "cistopic_qc_out")
if not os.path.exists(cistopic_qc_out):
    os.makedirs(cistopic_qc_out)

In [10]:
fragments_sub_dict = {}
regions_sub_dict = {}
for sample in fragments_dict:
    metadata_file = os.path.join(cistopic_qc_out, sample + "__metadata_bc.pkl")
    print(f"Checking if {metadata_file} exist...")
    if os.path.exists(metadata_file):
        print("\tMetadata exists! Skipping...")
    else:
        fragments_sub_dict[sample] = fragments_dict[sample]
        print("\tMetadata does not exist, adding to subdict to generate")
        regions_sub_dict = {x: region for x in sorted(fragments_sub_dict.keys())}

Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_2_cistopic/cistopic_qc_out/BIO_ddseq_1.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_2_cistopic/cistopic_qc_out/BIO_ddseq_2.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_2_cistopic/cistopic_qc_out/BIO_ddseq_3.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_2_cistopic/cistopic_qc_out/BIO_ddseq_4.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Checking if /dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/fixedcells_2_cistopic/cistopic_qc_out/BRO_mtscatac_1.FIXEDCELLS__metadata_bc.pkl exist...
	Metadata exists! Skipping...
Ch

In [11]:
ray.shutdown()

In [12]:
n_cores = 16
if fragments_sub_dict != {}:
    samples_sub = list(fragments_sub_dict.keys())
    blocks = [samples_sub[i : i + n_cores] for i in range(0, len(samples_sub), n_cores)]
    for samples_torun_in_block in blocks:
        fragments_sub_dict_block = {
            key: fragments_sub_dict[key] for key in samples_torun_in_block
        }
        regions_sub_dict_block = {
            key: regions_sub_dict[key] for key in samples_torun_in_block
        }

        metadata_bc_dict, profile_data_dict = compute_qc_stats(
            fragments_dict=fragments_sub_dict_block,
            tss_annotation=annotation,
            stats=[
                "barcode_rank_plot",
                "duplicate_rate",
                "insert_size_distribution",
                "profile_tss",
                "frip",
            ],
            label_list=None,
            path_to_regions=regions_sub_dict_block,
            n_cpu=n_cores,
            valid_bc=None,
            n_frag=10,
            n_bc=None,
            tss_flank_window=2000,
            tss_window=50,
            tss_minimum_signal_window=100,
            tss_rolling_window=10,
            # min_norm=0.2,
            remove_duplicates=True,
        )

        ray.shutdown()
        print(f"Dumping files in {cistopic_qc_out}...")
        for sample in sorted(metadata_bc_dict.keys()):
            metadata_bc_dict[sample]["sample_id"] = sample
            metadata_bc_dict[sample].index = [
                x + "___" + sample for x in list(metadata_bc_dict[sample].index)
            ]
            with open(
                os.path.join(cistopic_qc_out, f"{sample}__metadata_bc.pkl"), "wb"
            ) as f:
                pickle.dump(metadata_bc_dict[sample], f, protocol=4)

            with open(
                os.path.join(cistopic_qc_out, f"{sample}__profile_data.pkl"), "wb"
            ) as f:
                pickle.dump(profile_data_dict[sample], f, protocol=4)
else:
    print("All samples already processed.")

All samples already processed.


# Plot

Here, Otsu thresholding is used to find the right threshold for minimum fragments and minimum TSS enrichment. ddseq samples have a significantly higher noise floor than the other samples when it comes to fragment distribution. Therefore, the otsu algorithm is performed on all barcodes with a minimum of 300 fragments for the ddseq samples, and a minimum of 100 fragments for all the other samples. I tried to perform this filtering completely independent of sample/technique (e.g. using gaussian mixture modeling, Jenks natural breaks, or multiple step Otsu thresholding) but found that no solution worked perfectly for all samples.

This is regulated by the code below in qc_plots.py:
```
min_otsu_frags_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".")[0]
    tech = sample.split('_')[1]
    if tech == "ddseq":
        if sample == "BIO_ddseq_1":
            min_otsu_frags_dict[sample] = 600
        else:
            min_otsu_frags_dict[sample] = 300
    elif tech == "hydrop":
        min_otsu_frags_dict[sample] = 300
    else:
        min_otsu_frags_dict[sample] = 100
```

In [14]:
!cat ../0_resources/scripts/qc_plots.py

import kde
import pycisTopic
import glob
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import multiprocess as mp
from multiprocess import Pool
import pprint as pp

def histogram(array, nbins=100):
    """
    Draw histogram from distribution and identify centers.
    Parameters
    ---------
    array: `class::np.array`
            Scores distribution
    nbins: int
            Number of bins to use in the histogram
    Return
    ---------
    float
            Histogram values and bin centers.
    """
    array = array.ravel().flatten()
    hist, bin_edges = np.histogram(array, bins=nbins, range=None)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
    return hist, bin_centers


def threshold_otsu(array, nbins=100, min_value=100):
    """
    Apply Otsu threshold on topic-region distributions [Otsu, 1979].
    Parameters
    ---------
    array: `class::np.array`
            Array containing the region va

  
Since multiprocessing does not work with jupyter notebooks, run the following code in terminal:

```
mkdir plots_qc
mkdir selected_barcodes
SIF=../0_resources/cistopic_image/20220722_pycistopic.sif
singularity exec \
    --cleanenv \
    -H $PWD:/home \
    $SIF \
    python ../0_resources/scripts/qc_plots.py
```

And then open the plots

In [ ]:
metadata_bc_pkl_list = sorted(glob.glob("cistopic_qc_out/*metadata_bc.pkl"))
metadata_bc_pkl_path_dict = {}
for metadata_bc_pkl_path in metadata_bc_pkl_list:
    sample = metadata_bc_pkl_path.split("/")[-1].split("__metadata_bc.pkl")[0]
    metadata_bc_pkl_path_dict[sample] = metadata_bc_pkl_path

for sample in metadata_bc_pkl_path_dict.keys():
    if os.path.exists(f"selected_barcodes/{sample}_bc_passing_filters_otsu.pkl"):
        print(f"{sample} bc passing filters exists, printing img and skipping")
        display(Image(f"plots_qc/{sample}_qc_otsu.png"))
    else:
        print(
            f"{sample} bc passing filters does not exist yet, generate using qc_plots.py script!"
        )

# Write raw barcode file

Freemuxlet reads barcodes in the bam file. These barcodes (tag: DB) do not contain the `sample` suffix, which is included in the `*_bc_passing_filters_otsu.txt` files:

In [5]:
!head selected_barcodes/BIO_ddseq_1.FULL_bc_passing_filters_otsu.txt

head: cannot open 'selected_barcodes/BIO_ddseq_1.FULL_bc_passing_filters_otsu.txt' for reading: No such file or directory


In [6]:
for file in sorted(glob.glob("selected_barcodes/UCS*bc_passing_filters_otsu.txt")):
    print(file)
    sample = file.split("/")[-1].split("_bc_passing_filters_otsu.txt")[0]
    print(sample)
    df = pd.read_csv(file, header=None, index_col=None)

    with open(f"selected_barcodes/{sample}_bc_passing_filters_otsu.RAW.txt", "w") as fp:
        for x in df[0]:
            # write each item on a new line
            fp.write(x.split("___")[0] + "\n")
        print("Done")

selected_barcodes/UCS_ddseq_1.LIBDS_bc_passing_filters_otsu.txt
UCS_ddseq_1.LIBDS
Done
selected_barcodes/UCS_ddseq_2.LIBDS_bc_passing_filters_otsu.txt
UCS_ddseq_2.LIBDS
Done


In [7]:
!head selected_barcodes/BIO_ddseq_1.FULL_bc_passing_filters_otsu.RAW.txt

head: cannot open 'selected_barcodes/BIO_ddseq_1.FULL_bc_passing_filters_otsu.RAW.txt' for reading: No such file or directory
